In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from urllib.request import urlopen
from bs4 import BeautifulSoup
from urllib.request import FancyURLopener

# change fighter url 
# url = "https://www.sherdog.com/events/UFC-Fight-Night-101-Whittaker-vs-Brunson-54995"
url = "https://www.sherdog.com/fighter/Jon-Jones-27944"
fighter = url[32:-6]

# html = urlopen(url)
class MyOpener(FancyURLopener):
    version = 'My new User-Agent'

# spoof as browser to access url     
MyOpener.version
class MyOpener(FancyURLopener):
    version = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; it; rv:1.8.1.11) Gecko/20071127 Firefox/2.0.0.11'    
    
# open url and parse with bs4     
myopener = MyOpener()
page = myopener.open(url)
soup = BeautifulSoup(page, 'lxml')    
text = soup.get_text()

# find all table rows and assign to rows
rows = soup.find_all('tr')

str_cells = str(rows)
cleantext = BeautifulSoup(str_cells, "lxml").get_text()
    
    
import re

# further parse with regex
list_rows = []
for row in rows:
    cells = row.find_all('td')
    str_cells = str(cells)
    clean = re.compile('<.*?>')
    clean2 = (re.sub(clean, '',str_cells))
    list_rows.append(clean2)
    
# turn parsed text into df    
df = pd.DataFrame(list_rows)    

# delimit text and remove square brackets
df1 = df[0].str.split(',', expand=True)
df1[0] = df1[0].str.strip('[')
df1[5] = df1[5].str.strip(']')
df1.columns = df1.iloc[2]
# df1 = df1.reindex(df1.index.drop(2))
# df1.head(10)

# trim excess rows at the ends not pertaining to fight record
# will need to adjust dropping 0,1,2 for fighters who do not have an upcoming fight 
df2 = df1.drop([0,1,2,df.shape[0]-3,df.shape[0]-2,df.shape[0]-1],axis = 0)

cols=list(df2.columns)

# split out method and referee into 3 columns

df2[cols[3]] = df2[cols[3]].str.replace(')', '))')
methref = df2[cols[3]].str.rsplit(')', expand=True, n=1)
methref.columns = ['method','Referee']
fin = methref['method'].str.split('(',expand = True)
fin[1] = fin[1].str.strip(')')
fin.columns = ['Method','Finish']

# split up event and date

ev = df2[cols[2]].str.rsplit('/', expand=True)
ev['mth'] = ev[0].str[-4:]
ev[0] = ev[0].str[:-4]

#rearranging column order
ev.columns = ['Event','Day','Year','Month']
ev = ev[['Event','Month','Day','Year']]

#join all 4 tables together dropping excess event and method/referee columns that have been cleaned 
newtbl = pd.concat([df2.drop([cols[2], cols[3]],axis = 1) ,ev,fin,methref['Referee']],axis = 1)


# create fighter-name.csv
csvname = '.\\{}.csv'.format(fighter)
newtbl.to_csv(path_or_buf = csvname,index = False)

